#Automated Contract Clause Understanding and Risk Assessment with fine-tuned Legal-BERT and GPT-4

In [1]:
!pip install -U accelerate
!pip install -U transformers
!pip install openai==0.28
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 3.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [5]:
from datasets import load_dataset, Dataset
from google.colab import drive
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import torch
import openai
import numpy as np
import pandas as pd

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cuda


In [7]:
# Set up your OpenAI API key
openai.api_key = OPENAI_API_KEY

In [8]:
dataset = load_dataset("nguha/legalbench", 'cuad_audit_rights')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for nguha/legalbench contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/nguha/legalbench.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/6 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1216 [00:00<?, ? examples/s]

## Legal Text Classification: Audit Clause or Not


In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['answer', 'index', 'text', 'document_name'],
        num_rows: 6
    })
    test: Dataset({
        features: ['answer', 'index', 'text', 'document_name'],
        num_rows: 1216
    })
})

In [37]:
# Convert to DataFrame for easier handling
df_train = pd.DataFrame(dataset['test'])
df_test = pd.DataFrame(dataset['train'])

# Preprocess text
df_train['cleaned_text'] = df_train['text'].apply(lambda text: text.strip().lower())
df_test['cleaned_text'] = df_test['text'].apply(lambda text: text.strip().lower())

#To split the data better
df_combined = pd.concat([df_train, df_test])
df_combined.drop(columns=['index'])

# Shuffle the data
df_combined_shuffled = df_combined.sample(frac=1).reset_index(drop=True)
df_combined_shuffled.drop(columns=['index'])

# Split the data into training, validation, and test sets
train_data, test_data = train_test_split(df_combined_shuffled, test_size=0.2, stratify = df_combined_shuffled['answer'])
val_data, test_data = train_test_split(test_data, test_size=0.4, stratify = test_data['answer'])

print(f"Training set size: {train_data.shape}")
print(f"Validation set size: {val_data.shape}")
print(f"Test set size: {test_data.shape}")

Training set size: (977, 5)
Validation set size: (147, 5)
Test set size: (98, 5)


In [40]:
# Initialize the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('nlpaueb/legal-bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('nlpaueb/legal-bert-base-uncased', num_labels=2)
model.to(device)

# Tokenize the inputs
train_encodings = tokenizer(train_data['cleaned_text'].tolist(), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_data['cleaned_text'].tolist(), truncation=True, padding=True, max_length=512)

# Convert labels to tensor
train_labels = torch.tensor(train_data['answer'].apply(lambda x: 1 if x.lower() == "yes" else 0).tolist())
val_labels = torch.tensor(val_data['answer'].apply(lambda x: 1 if x.lower() == "yes" else 0).tolist())


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


###Fine-Tuning

In [ ]:
# Create dataset class
class LegalDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# Create datasets
train_dataset = LegalDataset(train_encodings, train_labels)
val_dataset = LegalDataset(val_encodings, val_labels)

In [60]:
def train_and_evaluate(train_texts, train_labels, val_texts, val_labels, model, tokenizer):

    # Tokenize the inputs
    train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=512)
    val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=512)

    # Convert labels to tensor
    train_labels = torch.tensor(train_labels.apply(lambda x: 1 if x.lower() == "yes" else 0).tolist())
    val_labels = torch.tensor(val_labels.apply(lambda x: 1 if x.lower() == "yes" else 0).tolist())

    # Create datasets
    train_dataset = LegalDataset(train_encodings, train_labels)
    val_dataset = LegalDataset(val_encodings, val_labels)

    training_args = TrainingArguments(
        output_dir='/results',
        num_train_epochs=3,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        eval_strategy="epoch"
    )

    trainer = Trainer(
        model = model,
        args = training_args,
        train_dataset = train_dataset,
        eval_dataset = val_dataset,
    )

    trainer.train()
    predictions = trainer.predict(val_dataset)
    preds = predictions.predictions.argmax(-1)
    labels = predictions.label_ids

    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    f1 = f1_score(labels, preds)

    return accuracy, precision, recall, f1

In [48]:
# Note:
# We used k-fold cross-validation to ensure that the model's performance is robust and generalizable.
# K-fold cross-validation splits the data into 'k' subsets, trains the model on 'k-1' subsets, and validates it on the remaining subset.
# This process is repeated 'k' times with different subsets as the validation set each time.
# By averaging the results, we get a more reliable estimate of the model's performance and reduce the risk of overfitting,
# as the model is validated on different data segments in each fold.

kf = StratifiedKFold(n_splits=5)
accuracies, precisions, recalls, f1s = [], [], [], []
texts = train_data['cleaned_text']
labels = train_data['answer']
i=0
for train_index, val_index in kf.split(texts, labels):
    train_texts = texts[texts.index.isin(train_index)]
    val_texts = texts[texts.index.isin(val_index)]
    train_labels = labels[labels.index.isin(train_index)]
    val_labels = labels[labels.index.isin(val_index)]

    #Initializing a new model
    model = BertForSequenceClassification.from_pretrained('nlpaueb/legal-bert-base-uncased', num_labels=2)
    tokenizer = BertTokenizer.from_pretrained('nlpaueb/legal-bert-base-uncased')
    model.to(device)

    accuracy, precision, recall, f1 = train_and_evaluate(train_texts, train_labels, val_texts, val_labels, model, tokenizer)

    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)

    model.save_pretrained('fine-tuned-legal-bert-fold'+str(i))
    tokenizer.save_pretrained('fine-tuned-legal-bert-fold'+str(i))
    i+=1

# Print average metrics
print(f"Average Accuracy: {sum(accuracies) / len(accuracies)}")
print(f"Average Precision: {sum(precisions) / len(precisions)}")
print(f"Average Recall: {sum(recalls) / len(recalls)}")
print(f"Average F1 Score: {sum(f1s) / len(f1s)}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.062200,0.047423
2,0.147000,0.036699
3,0.047700,0.100739


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.228600,0.100265
2,0.025200,0.026805
3,0.000600,0.054792


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.254500,0.094395
2,0.034400,0.049781
3,0.001200,0.079137


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.185300,0.126179
2,0.015100,0.046662
3,0.000600,0.133784


Average Accuracy: 0.9872256412463418
Average Precision: 0.9821596255150473
Average Recall: 0.9926673774530317
Average F1 Score: 0.9872990094514476


In [61]:
# Define the test function
def test_model(test_texts, test_labels, model_path, tokenizer_path):
    # Load the fine-tuned model and tokenizer
    model = BertForSequenceClassification.from_pretrained(model_path)
    tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
    model.to(device)

    # Tokenize the test texts
    test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=512)

    # Convert labels to tensor
    test_labels_tensor = torch.tensor(test_labels.apply(lambda x: 1 if x.lower() == "yes" else 0).tolist())

    # Create a test dataset
    test_dataset = LegalDataset(test_encodings, test_labels_tensor)

    # Create a DataLoader for the test dataset
    test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

    # Evaluate the model
    model.eval()
    preds = []
    labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels.extend(batch['labels'].cpu().numpy())
            outputs = model(input_ids, attention_mask=attention_mask)
            preds.extend(torch.argmax(outputs.logits, dim=-1).cpu().numpy())

    # Calculate metrics
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    f1 = f1_score(labels, preds)

    return accuracy, precision, recall, f1

In [57]:
# Load your test data
test_texts = test_data['cleaned_text']
test_labels = test_data['answer']

# Iterate over the saved models and evaluate them
for i in range(5):  # Assuming you have 5 models
    model_path = f'fine-tuned-legal-bert-fold{i}'
    tokenizer_path = f'fine-tuned-legal-bert-fold{i}'

    accuracy, precision, recall, f1 = test_model(test_texts, test_labels, model_path, tokenizer_path)
    print(f"Model {i} - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}")

accuracy, precision, recall, f1 = test_model(test_texts, test_labels,'nlpaueb/legal-bert-base-uncased', 'nlpaueb/legal-bert-base-uncased')
print(f"Model (untrained Legal-BERT) - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}")

Model 0 - Accuracy: 0.9795918367346939, Precision: 1.0, Recall: 0.9591836734693877, F1 Score: 0.9791666666666666
Model 1 - Accuracy: 0.9897959183673469, Precision: 0.98, Recall: 1.0, F1 Score: 0.98989898989899
Model 2 - Accuracy: 0.9897959183673469, Precision: 1.0, Recall: 0.9795918367346939, F1 Score: 0.9896907216494846
Model 3 - Accuracy: 0.9897959183673469, Precision: 1.0, Recall: 0.9795918367346939, F1 Score: 0.9896907216494846
Model 4 - Accuracy: 0.9795918367346939, Precision: 0.9607843137254902, Recall: 1.0, F1 Score: 0.98


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model (untrained Legal-BERT) - Accuracy: 0.47959183673469385, Precision: 0.4895833333333333, Recall: 0.9591836734693877, F1 Score: 0.6482758620689655


In [59]:
# drive.mount('/content/drive')
# for i in range(5):  # Assuming you have 5 models
#     model_path = f'fine-tuned-legal-bert-fold{i}'
#     tokenizer_path = f'fine-tuned-legal-bert-fold{i}'

#     model.save_pretrained('/content/drive/MyDrive/fine-tuned-legal-bert-5folds/'+ model_path)
#     tokenizer.save_pretrained('/content/drive/MyDrive/fine-tuned-legal-bert-5folds/'+ tokenizer_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [78]:
# Load model
model = BertForSequenceClassification.from_pretrained("/content/fine-tuned-legal-bert-fold1")
tokenizer = BertTokenizer.from_pretrained("/content/fine-tuned-legal-bert-fold1")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [79]:
# Function for classification using Legal-BERT
def classify_clause_legal_bert(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    return predictions.item()

# Define a test clause
test_clause = "Each party shall cause any subsidiary or other affiliate (including, without limitation, a subsidiary or other affiliate of the online group or skype group, as applicable) to grant to the other party the audit rights granted hereunder with respect to such other party."

# Get the combined result
response = classify_clause_legal_bert(test_clause)

# Print the combined result
print(response)

1


In [72]:
# # Set up OpenAI API key
# openai.api_key = OPENAI_API_KEY
# models_available = openai.Model.list()
# for model in models['data']:
#     print(model['id'])

whisper-1
babbage-002
dall-e-2
gpt-3.5-turbo-16k
tts-1-hd-1106
tts-1-hd
gpt-4-turbo-2024-04-09
gpt-4-0125-preview
gpt-4-turbo-preview
gpt-4-turbo
gpt-3.5-turbo-instruct-0914
gpt-4o
gpt-3.5-turbo-instruct
text-embedding-3-small
tts-1
gpt-4
text-embedding-3-large
gpt-4-1106-preview
gpt-4-0613
gpt-3.5-turbo-0125
tts-1-1106
gpt-3.5-turbo
dall-e-3
text-embedding-ada-002
davinci-002
gpt-3.5-turbo-1106
gpt-4o-2024-05-13


# Legal Text Risk Analysis

In [80]:
def run_riskAnalysis(clause):
    # Risk Analysis using GPT-4o
    risk_template = "You are a legal advisor. Identify any potential risks in the clauses given to you."
    prompt = clause
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": risk_template},
            {"role": "user", "content": prompt}
        ]
    )
    return response['choices'][0]['message']['content']

# Define a test clause
test_clause = "Each party shall cause any subsidiary or other affiliate (including, without limitation, a subsidiary or other affiliate of the online group or skype group, as applicable) to grant to the other party the audit rights granted hereunder with respect to such other party."

# Get the combined result
response = run_riskAnalysis(test_clause)

# Print the combined result
print(response)

Certainly, here are some potential risks in this clause:

1. **Scope and Definition of Affiliates/Subsidiaries**: The clause includes terms like "subsidiary" and "affiliate" without clearly defining them. Ambiguities in defining these entities could lead to disputes over whether an entity is indeed a subsidiary or affiliate and thus subject to this clause.

2. **Lack of Control Over Third Parties**: Each party might not have sufficient control over its affiliates and subsidiaries to compel them to grant audit rights as required by the clause. This lack of control can lead to non-compliance and breach of contract.

3. **Unlimited and Undefined Audit Rights**: The clause vaguely refers to "audit rights granted hereunder" without specifying the scope, duration, and boundaries of these rights. This could lead to intrusive audits and potential disputes over privacy and confidentiality.

4. **Potential conflicts with existing agreements**: Affiliates and subsidiaries may have existing agreem

In [81]:
# Function to run the prompt using the ChatCompletion endpoint
def run_gpt_integration(classification_label, risk_analysis, clause):
    prompt = (
        f"Here is a contract clause that has been classified as '{classification_label}':\n\n"
        f"'{clause}'\n\n"
        f"The potential risks identified in this clause are:\n{risk_analysis}\n\n"
    )
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a legal advisor. Please provide an integrated, cohesive explanation of this clause, its classification, and the identified risks. Provide the respone in the following template:"},
            {"role": "user", "content": prompt}
        ]
    )
    return response['choices'][0]['message']['content']

In [82]:
# Define a combined function
def classify_and_analyze_clause(clause):
    classification_result = classify_clause_legal_bert(clause)
    classification_label = "Audit Clause" if classification_result == 1 else "Not an Audit Clause"
    risk_analysis = run_riskAnalysis(clause)
    integrated_response = run_gpt_integration(classification_label, risk_analysis, clause)
    return integrated_response

# Define a test clause
test_clause = "Each party shall cause any subsidiary or other affiliate (including, without limitation, a subsidiary or other affiliate of the online group or skype group, as applicable) to grant to the other party the audit rights granted hereunder with respect to such other party."

# Get the combined result
response = classify_and_analyze_clause(test_clause)

# Print the combined result
print(response)

### Clause Explanation and Risk Assessment

**Clause:**
'Each party shall cause any subsidiary or other affiliate (including, without limitation, a subsidiary or other affiliate of the online group or Skype group, as applicable) to grant to the other party the audit rights granted hereunder with respect to such other party.'

**Clause Classification**: Audit Clause

### Explanation:
This clause mandates that both parties ensure their subsidiaries and other affiliates provide the other party with the same audit rights that the primary parties themselves have agreed to. The clause extends audit rights to include any subsidiary or affiliate, specifically mentioning those associated with "online groups" or "Skype groups."

### Potential Risks:
1. **Ambiguity in Scope**: The lack of clear definitions for "online group" and "Skype group" can create confusion over which entities fall under these categories, possibly leading to disputes over the breadth and application of audit rights.

2. **C